In [23]:
from dotenv import load_dotenv

load_dotenv()
import nest_asyncio
import os

nest_asyncio.apply()
import json
import base64

In [24]:
# Load json from data/data.json

with open("data/data.json") as f:
    data = json.load(f)
print(data["script"])

# Constants:
img_urls = ["""https://cdn.discordapp.com/attachments/1212193794494042202/1223728760948523190/jas1.jpg?ex=661ae938&is=66087438&hm=a2661c174ebec9afba3265e3eb6bb86832f732f66d88195a5faa293a09839654&""", """https://cdn.discordapp.com/attachments/1221634008656642048/1223822939875704904/IMG_2084.jpg?ex=661b40ee&is=6608cbee&hm=ec453c6196ea2b6fc3bfa3e9fcb7f14a438f955f69198d937ea079da55a2afd6&"""]
voice_ids = ["jA3XNbtepbUtYOz5Q5bI", "UCHQN7CfyPSQTpOZoSvi"]
bearer = os.environ.get("DID_API_KEY")
encoded = base64.b64encode(bearer.encode("utf-8")).decode("utf-8")
eleven = os.environ.get("ELEVENLABS_API_KEY")

<speak>
Have you ever heard of a <emphasis>trie</emphasis>? No, not the verb "try", I'm talking about the <emphasis>data structure</emphasis> called a trie, spelled T-R-I-E. <break time="1s"/>

Imagine you're organizing a massive library, but instead of books, you're dealing with <emphasis>words</emphasis>. A trie is like a tree of arrays that allows you to store and retrieve these words at lightning-fast speeds. <break time="2s"/>

Picture each letter of the alphabet having its own shelf. To store a word, you simply place it on the shelf corresponding to its first letter. Then, <emphasis>each subsequent letter</emphasis> gets its own sub-shelf branching off from the previous one. <break time="2s"/>

The magic happens when you're looking for a specific word. <emphasis>No matter how many words are in the library, you can find any word in the same amount of time</emphasis> - just by following the path created by its letters! <break time="2s"/>

So the next time you're typing on your phon

In [25]:
import asyncio
import aiohttp

async def post_talk(script):
    url = "https://api.d-id.com/talks"
    headers = {
        'accept': 'application/json',
        'authorization': f'Basic {encoded}',
        'Content-Type': 'application/json',
        'x-api-key-external': json.dumps({"elevenlabs": eleven})
    }
    data = {
        "script": {
            "type": "text",
            "subtitles": "false",
            "provider": {
                "type": "elevenlabs",
                "voice_id": voice_ids[1],
                "voice_config": {"stability": 0.3, "similarity_boost": 1},
            },
            "model_id": "eleven_multilingual_v2",
            "input": script,
            "ssml": True
        },
        "config": {"fluent": "false", "pad_audio": "0.0"},
        "source_url": img_urls[1],
    }

    async with aiohttp.ClientSession() as session:
        async with session.post(url, headers=headers, json=data) as response:
            # Check if the request was successful
            if response.status == 201:
                # Process the response here
                response_data = await response.json()
                id = response_data["id"]

            else:
                # Handle request errors here
                print(f"Failed to post data, status code: {response.status}")
                print(await response.text())
                return ""

        # Loop to check the status
        while True:
            async with session.get(f"{url}/{id}", headers=headers) as status_response:
                if status_response.status == 200:
                    status_data = await status_response.json()
                    if status_data["status"] == "done":
                        result_url = status_data["result_url"]
                        break
                    else:
                        # Wait for some time before checking the status again
                        await asyncio.sleep(5)
                else:
                    print(f"Failed to get data, status code: {status_response.status}")
                    return ""

        # Download the video
        async with session.get(result_url) as video_response:
            if video_response.status == 200:
                mp4_data = (
                    await video_response.read()
                )  # This is the binary data of the MP4 file
                return mp4_data
            else:
                print(f"Failed to download video, status code: {video_response.status}")
                return ""

In [26]:
video = asyncio.run(post_talk(data["script"]))

In [ ]:
# Save video
with open("data/headshot.mp4", "wb") as f:
    f.write(video)